In [3]:
import pandas as pd
import numpy as np
import quandl

quandl.ApiConfig.api_key = "zFtyXTJz6DTqqdvEy2gi"

In [13]:
data = quandl.get_table('WIKI/PRICES', qopts = { 'columns': ['ticker', 'date', 'close'] }, ticker = ['DBA'], date = { 'gte': '2012-01-01', 'lte': '2018-04-13' })

In [14]:
len(data)

0

In [32]:
#continuous futures W1 is front month
#W1 may18 prices start 3/15/18..futures trade until biz day before 15th of settle month
data = quandl.get('CHRIS/CME_W7')

In [33]:
data.to_csv('CME_W7.csv')

In [35]:
data.head()

,Open,High,Low,Last,Change,Settle,Volume,Previous Day Open Interest
Date,,,,,,,,
2014-12-24,631.5,631.5,631.5,631.0,17.0,631.0,2.0,144.0
2014-12-26,631.0,631.0,631.0,631.0,0.0,631.0,0.0,146.0
2014-12-29,634.5,634.5,634.5,634.5,3.5,634.5,3.0,146.0
2014-12-30,629.0,629.0,625.0,624.5,10.0,624.5,10.0,147.0
2014-12-31,613.0,613.0,613.0,613.0,11.5,613.0,0.0,151.0


In [34]:
data.iloc[14780:,]

,Open,High,Low,Last,Change,Settle,Volume,Previous Day Open Interest
Date,,,,,,,,


In [60]:
#individual contract (July 2018)
#data = quandl.get('CME/WN2018')
#data = quandl.get('CME/WK2018')  #may2018'
data = quandl.get('CME/WH2018')  #mar2018


In [61]:
data.iloc[635:,[3]]

,Last
Date,
2018-01-22,425.00
2018-01-23,421.50
2018-01-24,432.50
2018-01-25,434.75
2018-01-26,441.25
2018-01-29,448.75
2018-01-30,457.00
2018-01-31,451.50
2018-02-01,450.75
